# Transfer learning: PNASNet-5-Large

## The code that produces the .csv file is at the very bottom.

In [1]:
import torch
import numpy as np

from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.autograd import Variable
import PIL.Image as Image
from torchvision import models
import time
import os
import pretrainedmodels

In [2]:
normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )

train_transform = transforms.Compose([
        #transforms.Resize(256),
        transforms.RandomRotation(45),
        transforms.RandomResizedCrop(331),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
        ])

valid_transform = transforms.Compose([
        #transforms.Resize(256),
        transforms.CenterCrop(331),
        transforms.ToTensor(),
            normalize,
        ])


train_dataset = ImageFolder(
        root='./bird_dataset/train_images/', 
        transform=train_transform
    )

valid_dataset = ImageFolder(
        root='./bird_dataset/val_images/', 
        transform=valid_transform
    )

In [3]:
batch_size = 4
num_workers = 1  #1 if cuda
pin_memory = True  #true if cuda
shuffle = True

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=shuffle,
        num_workers=num_workers, pin_memory=pin_memory,
    )

valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, shuffle=shuffle,
        num_workers=num_workers, pin_memory=pin_memory,
    )

In [4]:
cuda_avail = torch.cuda.is_available()
model_ft = pretrainedmodels.__dict__['pnasnet5large'](num_classes=1000, pretrained='imagenet')
num_ftrs = model_ft.last_linear.in_features
model_ft.fc = nn.Linear(num_ftrs, 20)      
 
if cuda_avail:                                
    model_ft = model_ft.cuda()       
loss_fn = nn.CrossEntropyLoss()          


optimizer = SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def save_models(epoch):
    torch.save(model_ft.state_dict(), "pnasnet5largemodel_{}.model".format(epoch))
    print("Checkpoint saved")

def test():
    model_ft.eval()
    test_acc = 0.0
    for data in valid_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model_ft(images)
        _,prediction = torch.max(outputs.data, 1)
        test_acc += (prediction == labels).sum().item()
        

    test_acc = test_acc / 103

    return test_acc

best_acc_train = 0.0
best_acc_test = 0.0

def train(num_epochs):
    global best_acc_train
    global best_acc_test
    for epoch in range(num_epochs):
        model_ft.train()
        train_acc = 0.0
        train_loss = 0.0
        for data in train_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model_ft(images)
            loss = loss_fn(outputs,labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.cpu().item() * images.size(0)
            _, prediction = torch.max(outputs.data, 1)
            
            train_acc += (prediction == labels).sum().item()

        train_acc = train_acc / 1082
        train_loss = train_loss / 1082

        test_acc = test()

        if (test_acc >= best_acc_test) and (train_acc >= best_acc_train):
            save_models(epoch)
            best_acc_test = test_acc
            best_acc_train = train_acc


        # Print the metrics
        print("Epoch {}, Train Accuracy: {} , TrainLoss: {} , Test Accuracy: {}".format(epoch, train_acc, train_loss,test_acc))


In [6]:
optimizer = SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
train(10)

Checkpoint saved
Epoch 0, Train Accuracy: 0.3567467652495379 , TrainLoss: 2.8480817597366306 , Test Accuracy: 0.8155339805825242
Checkpoint saved
Epoch 1, Train Accuracy: 0.6487985212569316 , TrainLoss: 1.1572289713649786 , Test Accuracy: 0.912621359223301
Epoch 2, Train Accuracy: 0.7476894639556377 , TrainLoss: 0.8192005637833459 , Test Accuracy: 0.8932038834951457
Epoch 3, Train Accuracy: 0.7929759704251387 , TrainLoss: 0.6637907535002985 , Test Accuracy: 0.8932038834951457
Epoch 4, Train Accuracy: 0.7781885397412199 , TrainLoss: 0.6991794655812206 , Test Accuracy: 0.883495145631068
Epoch 5, Train Accuracy: 0.822550831792976 , TrainLoss: 0.62207459169483 , Test Accuracy: 0.883495145631068
Epoch 6, Train Accuracy: 0.8234750462107209 , TrainLoss: 0.5635375143639922 , Test Accuracy: 0.8932038834951457
Epoch 7, Train Accuracy: 0.8604436229205176 , TrainLoss: 0.43918722085723594 , Test Accuracy: 0.9029126213592233
Epoch 8, Train Accuracy: 0.8558225508317929 , TrainLoss: 0.4437306910037113

In [7]:
optimizer = SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
train(5)

Epoch 0, Train Accuracy: 0.8743068391866913 , TrainLoss: 0.4503294033396046 , Test Accuracy: 0.912621359223301
Epoch 1, Train Accuracy: 0.8558225508317929 , TrainLoss: 0.4449432024012652 , Test Accuracy: 0.883495145631068
Epoch 2, Train Accuracy: 0.8853974121996303 , TrainLoss: 0.39059313564335796 , Test Accuracy: 0.9029126213592233
Epoch 3, Train Accuracy: 0.8881700554528651 , TrainLoss: 0.3826705973161567 , Test Accuracy: 0.8640776699029126
Epoch 4, Train Accuracy: 0.8964879852125693 , TrainLoss: 0.3400879051680926 , Test Accuracy: 0.9029126213592233


In [11]:
optimizer = SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
train(5)

Epoch 0, Train Accuracy: 0.8826247689463955 , TrainLoss: 0.40689175054017807 , Test Accuracy: 0.8932038834951457
Checkpoint saved
Epoch 1, Train Accuracy: 0.899260628465804 , TrainLoss: 0.3383355352221928 , Test Accuracy: 0.9029126213592233
Checkpoint saved
Epoch 2, Train Accuracy: 0.9066543438077634 , TrainLoss: 0.2886585149218547 , Test Accuracy: 0.912621359223301
Epoch 3, Train Accuracy: 0.8974121996303143 , TrainLoss: 0.3266038123428711 , Test Accuracy: 0.9223300970873787
Epoch 4, Train Accuracy: 0.8909426987060998 , TrainLoss: 0.3874747722294326 , Test Accuracy: 0.9029126213592233


In [12]:
model_ft.load_state_dict(torch.load('./densenetmodel_2.model'))
optimizer = SGD(model_ft.parameters(), lr=0.000001, momentum=0.9)
train(5)

Epoch 0, Train Accuracy: 0.88909426987061 , TrainLoss: 0.3512711357497464 , Test Accuracy: 0.9320388349514563
Epoch 1, Train Accuracy: 0.8909426987060998 , TrainLoss: 0.36845839089695054 , Test Accuracy: 0.9320388349514563
Epoch 2, Train Accuracy: 0.9038817005545287 , TrainLoss: 0.3142694285528497 , Test Accuracy: 0.9223300970873787
Epoch 3, Train Accuracy: 0.890018484288355 , TrainLoss: 0.35854611908001294 , Test Accuracy: 0.8932038834951457
Epoch 4, Train Accuracy: 0.9048059149722736 , TrainLoss: 0.30387497741501784 , Test Accuracy: 0.9223300970873787


In [18]:
#optimizer = SGD(model_ft.parameters(), lr=0.000001, momentum=0.9)
train(20)

Epoch 0, Train Accuracy: 0.8974121996303143 , TrainLoss: 0.3278011068178413 , Test Accuracy: 0.9223300970873787
Checkpoint saved
Epoch 1, Train Accuracy: 0.9121996303142329 , TrainLoss: 0.31096666048723315 , Test Accuracy: 0.912621359223301
Epoch 2, Train Accuracy: 0.9066543438077634 , TrainLoss: 0.31288091444484406 , Test Accuracy: 0.9223300970873787
Epoch 3, Train Accuracy: 0.8937153419593346 , TrainLoss: 0.3416318161870988 , Test Accuracy: 0.9223300970873787
Epoch 4, Train Accuracy: 0.9066543438077634 , TrainLoss: 0.31887432651907593 , Test Accuracy: 0.9223300970873787
Checkpoint saved
Epoch 5, Train Accuracy: 0.9140480591497228 , TrainLoss: 0.30257758635910514 , Test Accuracy: 0.9223300970873787
Epoch 6, Train Accuracy: 0.9038817005545287 , TrainLoss: 0.3014515944637785 , Test Accuracy: 0.912621359223301
Epoch 7, Train Accuracy: 0.9094269870609981 , TrainLoss: 0.3104034254599411 , Test Accuracy: 0.9320388349514563
Epoch 8, Train Accuracy: 0.9121996303142329 , TrainLoss: 0.282875701

In [20]:
train(20)

Epoch 0, Train Accuracy: 0.9195933456561922 , TrainLoss: 0.27247023097688744 , Test Accuracy: 0.941747572815534
Epoch 1, Train Accuracy: 0.911275415896488 , TrainLoss: 0.2909758496416695 , Test Accuracy: 0.9320388349514563
Epoch 2, Train Accuracy: 0.9011090573012939 , TrainLoss: 0.33707578785979153 , Test Accuracy: 0.9320388349514563
Epoch 3, Train Accuracy: 0.9177449168207024 , TrainLoss: 0.2847066670380767 , Test Accuracy: 0.9320388349514563
Epoch 4, Train Accuracy: 0.9020332717190388 , TrainLoss: 0.30952808729161174 , Test Accuracy: 0.9320388349514563
Epoch 5, Train Accuracy: 0.9057301293900185 , TrainLoss: 0.3113683843348251 , Test Accuracy: 0.9320388349514563
Epoch 6, Train Accuracy: 0.9020332717190388 , TrainLoss: 0.3214926702037536 , Test Accuracy: 0.9029126213592233
Epoch 7, Train Accuracy: 0.8974121996303143 , TrainLoss: 0.32400799899356864 , Test Accuracy: 0.9223300970873787
Epoch 8, Train Accuracy: 0.9001848428835489 , TrainLoss: 0.32729361546458247 , Test Accuracy: 0.932038

In [21]:
model_ft.load_state_dict(torch.load('./densenetmodel_10.model'))

In [25]:
train(20)

Epoch 0, Train Accuracy: 0.9085027726432532 , TrainLoss: 0.29867289819911313 , Test Accuracy: 0.9320388349514563
Epoch 1, Train Accuracy: 0.9057301293900185 , TrainLoss: 0.3029244660009077 , Test Accuracy: 0.9320388349514563
Epoch 2, Train Accuracy: 0.9094269870609981 , TrainLoss: 0.2873449713377327 , Test Accuracy: 0.9223300970873787
Epoch 3, Train Accuracy: 0.9177449168207024 , TrainLoss: 0.2502479751537556 , Test Accuracy: 0.9223300970873787
Epoch 4, Train Accuracy: 0.9085027726432532 , TrainLoss: 0.29572015240540567 , Test Accuracy: 0.9320388349514563
Epoch 5, Train Accuracy: 0.9020332717190388 , TrainLoss: 0.310632077691295 , Test Accuracy: 0.912621359223301
Epoch 6, Train Accuracy: 0.911275415896488 , TrainLoss: 0.2909331000005472 , Test Accuracy: 0.912621359223301
Checkpoint saved
Epoch 7, Train Accuracy: 0.9232902033271719 , TrainLoss: 0.26660372413240385 , Test Accuracy: 0.9320388349514563
Epoch 8, Train Accuracy: 0.9205175600739371 , TrainLoss: 0.2732248244576445 , Test Accur

In [26]:
model_ft.load_state_dict(torch.load('./pnasnet5largemodel_7.model'))

In [27]:
test_dir = './bird_dataset/test_images/mistery_category'
output_file = open("testdensenet.csv", "w")
output_file.write("Id,Category\n")

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

for f in os.listdir(test_dir):
    if 'jpg' in f:
        data = valid_transform(pil_loader(test_dir + '/' + f))
        data = data.view(1, data.size(0), data.size(1), data.size(2))
        if cuda_avail:
            data = data.cuda()
        model_ft.eval()
        output = model_ft(data)
        pred = output.data.max(1, keepdim=True)[1]
        output_file.write("%s,%d\n" % (f[:-4], pred))

output_file.close()

In [14]:
torch.save(model_ft.state_dict(), "pnaslarge_0.81290_.model")